# Partial Configuration Identification


We consider performance analysis in highly variable systems, such as x264, with a large number of configurations options and input features, and multiple performance metrics.
Our goal in this work to identify partial stable configurations from a set of measurements.
A stable configuration is a configuration that is consistently high performing, indicated by its presence in the Pareto front of configurations for each of the videos in the measurement set.
In the ideal case, we identify a single configuration that is present in every Pareto front, i.e. it is never dominated by another configuration. This would be the premier configuration to test for every new input.
Otherwise, we can identify the subset of configurations options that most frequently occur together and propose these as a partial configuration. The extensions of this partial configuration can then form a reduced set of candidate configurations to consider for evaluation/application. The extensions can be similarly determined from the obtained measurements and potentially be weighted by their expected performance resp. their influence on certain performance metrics.

We raise the following research questions:
- Do stable configurations exist? Are they partial or full configurations?
- Do the best configurations share common parameters?
- What is the largest common set of parameters with minimum quality?
- How much do the results vary if we further partition the data, e.g. by video category?
   
Technically speaking, we can apply techniques from data mining, e.g. frequent itemset mining, to determine the core partial configurations, but we do not have to rely on complex statistical machine learning techniques, which makes this approach simple and interpretable.
    
We present an in-depth analysis of our approach on a large-scale dataset of video encodings using the x264 video encoder.
Additionally, we confirm the applicability of our approach on seven others configurable systems (Luc's dataset: gcc, imagemagick, lingeling, nodeJS, poppler, SQLite, xz).

First we import some packages.
`common` is our shared library for shared functions like loading, ranking, etc.

In [1]:
import pandas as pd
import numpy as np
from common import load_data, pareto_rank

from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import fpgrowth, fpmax

We load the data for one system, here `x264`.

In [2]:
(
    perf_matrix,
    input_features,
    config_features,
    all_performances,
    input_preprocessor,
    config_preprocessor,
) = load_data(
    system="x264", data_dir="../data", input_properties_type="tabular"
)

In [3]:
config_features

,cabac,ref,subme,mixed_ref,me_range,trellis,8x8dct,fast_pskip,chroma_qp_offset,bframes,...,analyse,me,direct,deblock,b_adapt,b_pyramid,open_gop,rc_lookahead,scenecut,weightb
configurationID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,16,0,0,1,0,0,...,0:0,dia,NaN,0:0:0,NaN,NaN,NaN,NaN,0.0,NaN
2,1,1,1,0,16,0,1,1,0,3,...,0x3:0x3,dia,auto,1:0:0,1.0,2.0,0.0,NaN,40.0,1.0
3,1,1,2,0,16,0,1,1,0,3,...,0x3:0x113,hex,auto,1:0:0,1.0,2.0,0.0,10.0,40.0,1.0
4,1,2,4,0,16,0,1,1,0,3,...,0x3:0x113,hex,auto,1:0:0,1.0,2.0,0.0,20.0,40.0,1.0
5,1,2,6,1,16,1,1,1,-2,3,...,0x3:0x113,hex,auto,1:0:0,1.0,2.0,0.0,30.0,40.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,1,16,7,1,24,2,1,0,-2,16,...,0x3:0x133,tesa,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0
198,1,16,11,1,24,2,1,0,-2,16,...,0x3:0x133,hex,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0
199,1,16,11,1,24,2,1,0,-2,16,...,0x3:0x3,tesa,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0


## Pareto Front Calculation

We take the matrix of all measurements and calculate the Pareto ranks for each configuration per input. For this, we consider all measured performances, but we can change this to any subset.

In [4]:
icm = (
    perf_matrix[["inputname", "configurationID"] + all_performances]
    .sort_values(["inputname", "configurationID"])
    .set_index(["inputname", "configurationID"])
)
icm["ranks"] = icm.groupby("inputname", group_keys=False).apply(pareto_rank)

In [5]:
subdf = icm[icm.ranks < 3]
subdf

usertime  systemtime  etime  cpu  \
inputname            configurationID                                     
Animation_1080P-01b3 1                    7.94        1.36   2.14  434   
                     2                   11.38        1.46   2.62  489   
                     3                   14.29        1.70   2.80  571   
                     4                   17.10        1.81   2.86  661   
                     5                   16.81        1.46   2.83  644   
...                                        ...         ...    ...  ...   
Vlog_720P-6d56       169                 61.06        1.42   8.62  724   
                     173                 20.73        1.06   2.57  847   
                     176                101.75        3.06  16.33  641   
                     177                 47.78        2.65  10.30  489   
                     180                 63.97        0.96   8.92  727   

                                         fps      kbs  ranks  
inputname            configurationID                          
Animation_1080P-01b3 1               -375.22   161.07      1  
                     2               -302.51   103.42      1  
                     3               -281.46    63.70      1  
                     4               -271.22    73.24      2  
                     5               -273.05    70.16      1  
...                                      ...      ...    ...  
Vlog_720P-6d56       169              -72.32  4054.07      1  
                     173             -274.69  4538.05      2  
                     176              -37.56  4269.50      2  
                     177              -60.17  4480.01      1  
                     180              -69.85  4369.10      1  

[120675 rows x 7 columns]

In [6]:
dataset = subdf.join(config_features).reset_index()[
    config_features.columns
]

In [7]:
# To use mlxtend's itemset mining, we must convert the dataset to onehot encoding
enc = OneHotEncoder(
    min_frequency=1,
    handle_unknown="infrequent_if_exist",
    sparse_output=False,
)
enc.fit(dataset)
col_names = enc.get_feature_names_out()
onehot_data = enc.transform(dataset)
print(f"One-hot encoded dataset has {len(col_names)} columns ({dataset.shape[1]} before)")

One-hot encoded dataset has 78 columns (24 before)


In [8]:
df = pd.DataFrame(onehot_data, columns=col_names, dtype=np.bool_)
df

,cabac_0,cabac_1,ref_1,ref_2,ref_3,ref_5,ref_7,ref_8,ref_16,subme_0,...,rc_lookahead_30.0,rc_lookahead_40.0,rc_lookahead_50.0,rc_lookahead_60.0,rc_lookahead_nan,scenecut_0.0,scenecut_40.0,scenecut_nan,weightb_1.0,weightb_nan
0,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,True,True,False,False,False,True
1,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,True,False
2,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
4,False,True,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120670,False,True,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,True,False,True,False
120671,False,True,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,True,False
120672,False,True,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,True,False
120673,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


## FP-Growth
Extracting frequent itemsets from a dataset, i.e. which items to appear together most commonly.

In [9]:
fpgrowth(df, min_support=0.6, use_colnames=True).sort_values("support")

,support,itemsets
41,0.600447,"(trellis_0, mixed_ref_0)"
44,0.600447,"(trellis_0, qpmax_69, mixed_ref_0)"
85,0.601807,"(qpmax_69, scenecut_40.0, open_gop_0.0, b_pyra..."
84,0.601807,"(weightb_1.0, open_gop_0.0, b_pyramid_2.0, sce..."
83,0.601807,"(weightb_1.0, qpmax_69, b_pyramid_2.0, scenecu..."
...,...,...
2,0.867819,(fast_pskip_1)
15,0.867819,"(qpmax_69, fast_pskip_1)"
1,0.877555,(me_range_16)
14,0.877555,"(qpmax_69, me_range_16)"


## FP-Max 

FP-Max is a variant of FP-Growth, which focuses on obtaining maximal itemsets. 
An itemset X is said to maximal if X is frequent and there exists no frequent super-pattern containing X. 
In other words, a frequent pattern X cannot be sub-pattern of larger frequent pattern to qualify for the definition maximal itemset.

In [10]:
fpmax(df, min_support=0.6, use_colnames=True).sort_values("support")

,support,itemsets
3,0.600447,"(trellis_0, qpmax_69, mixed_ref_0)"
1,0.601807,"(qpmax_69, scenecut_40.0, open_gop_0.0, b_pyra..."
8,0.605801,"(weightb_1.0, open_gop_0.0, me_range_16, qpmax..."
4,0.605933,"(qpmax_69, mixed_ref_0, chroma_qp_offset_0)"
5,0.608486,"(qpmax_69, fast_pskip_1, mixed_ref_0)"
7,0.618239,"(8x8dct_1, open_gop_0.0, weightb_1.0, qpmax_69)"
9,0.618463,"(trellis_0, qpmax_69, fast_pskip_1, me_range_16)"
0,0.619308,"(weightb_1.0, open_gop_0.0, b_adapt_1.0, qpmax..."
10,0.632003,"(trellis_0, qpmax_69, fast_pskip_1, chroma_qp_..."
2,0.647541,"(qpmax_69, aq-mode_1)"


## Open Questions

- How do the itemsets change if we partition the data? E.g. by input features like category
- How do the itemsets change if we vary the pareto rank?
- What will we observe on the other datasets?
- What do we take away from the results?
- How can it be presented?

Orthogonal to the question of "What configuration options appear the most in the Pareto fronts" is the question of "What full configurations are the most common in the Pareto fronts":
This we should be able to answer by looking at the largest itemset we can find; here it's interesting to compare it's length to the number of configuration options.